In [1]:
import pandas as pd
import numpy as np
import glob
import pydicom
import cv2
import os, os.path as osp

from scipy.ndimage.interpolation import zoom
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pylab as plt
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

# import cv2
import glob
import pickle

In [341]:
# from IPython.display import FileLink
# FileLink(r'./dd.pkl')

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

device

device(type='cuda')

In [3]:
!ls ../input/rsna-str-pulmonary-embolism-detection/

sample_submission.csv  test  test.csv  train  train.csv


In [4]:
sample_df = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/sample_submission.csv')
df = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/test.csv')

dicom_folders = list(('../input/rsna-str-pulmonary-embolism-detection/test/' + df.StudyInstanceUID + '/'+ df.SeriesInstanceUID).unique())

In [5]:
len(dicom_folders)

650

In [6]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

test_ds_trans = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Grayscale(num_output_channels=3),
                                transforms.Resize(224),
                                transforms.ToTensor(),
                                
#                                 transforms.CenterCrop(224),
                                normalize
])

invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

In [8]:
sample_df

,id,label
0,df06fad17bc3_negative_exam_for_pe,0.5
1,c8039e7f9e63_negative_exam_for_pe,0.5
2,761f6f1a9f5b_negative_exam_for_pe,0.5
3,c8db5b1f6b56_negative_exam_for_pe,0.5
4,462e805da1f1_negative_exam_for_pe,0.5
...,...,...
152698,5f34e0c61c00,0.5
152699,ccaa309b60da,0.5
152700,a274c8d0916e,0.5
152701,a702de2c99c6,0.5


In [9]:
df.StudyInstanceUID.unique()

array(['df06fad17bc3', 'c8039e7f9e63', '761f6f1a9f5b', 'c8db5b1f6b56',
       '462e805da1f1', '7f6fb39566ed', 'b44cbf5371f2', '62dfc5f411e8',
       '1870d65d0f6a', '26135e3b3b30', 'acd8f049599c', 'cd367839ad5b',
       '79981becb2f8', 'b1c8e2c38aae', 'e8638ffea9f2', '274d5e835510',
       'a476d4c317c7', '2c45c4483cb7', '85e03629acad', '16997c0ce2d9',
       '6642d224880c', '79d2d31a771d', '684b1e867538', '7535273c83a2',
       '1267d9b650e1', '6fa5a9c4f2ce', 'a35eeffdce0c', '45bc2708a044',
       'ba8b55006edc', 'd953b98f0ae1', '9458a460b374', '018dbbbea69e',
       '950659d7ad56', 'c6cd915b6d85', '25399bedbfa8', '8edac04a0d4e',
       'd2909eb50025', '0d5f3216ec1a', '3b27216432d3', '14539f5d608e',
       '0cfc61e46f28', '36aada358977', '072d0d0261a7', 'f10890f301c6',
       '2295de87298e', '1e0c7650e334', '8e56848dfe38', '71c1460ed14c',
       '95137d0b4ab8', '4543d932f3dd', 'cccf5a39cdaf', '16acac6e5a69',
       '67ee6e1b97c7', '44f2e5214cda', '0a780dd0d52b', 'e8752f23ad82',
      

In [10]:
def load_dicom_array(f):
    dicom_files = glob.glob(osp.join(f, '*.dcm'))
    dicoms = [pydicom.dcmread(d) for d in dicom_files]
    M = float(dicoms[0].RescaleSlope)
    B = float(dicoms[0].RescaleIntercept)
    # Assume all images are axial
    z_pos = [float(d.ImagePositionPatient[-1]) for d in dicoms]
    dicoms = np.asarray([d.pixel_array for d in dicoms])
    dicoms = dicoms[np.argsort(z_pos)]
    dicoms = dicoms * M
    dicoms = dicoms + B
    return dicoms, np.asarray(dicom_files)[np.argsort(z_pos)]


def window(img, WL=50, WW=350):
    upper, lower = WL+WW//2, WL-WW//2
    X = np.clip(img.copy(), lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    X = (X*255.0).astype('uint8')
    return X


def save_array(X, save_dir, file_names):
    for ind, img in enumerate(X):
        savefile = osp.join(save_dir, file_names[ind])
        if not osp.exists(osp.dirname(savefile)): 
            os.makedirs(osp.dirname(savefile))
        _ = cv2.imwrite(osp.join(save_dir, file_names[ind]), img)


def edit_filenames(files):
    dicoms = [f"{ind:04d}_{f.split('/')[-1].replace('dcm','jpg')}" for ind,f in enumerate(files)]
    series = ['/'.join(f.split('/')[-3:-1]) for f in files]
#     print(files[0], dicoms[0], series[0])
    return [osp.join(s,d) for s,d in zip(series, dicoms)]


class Lungs(Dataset):
    def __init__(self, dicom_folders):
        self.dicom_folders = dicom_folders
#         self.trans = trans
    def __len__(self): return len(self.dicom_folders)
    def get(self, i):
        return load_dicom_array(self.dicom_folders[i])
    def __getitem__(self, i):
        try:
            return self.get(i)
        except Exception as e:
            print(e)
            return None


# SAVEDIR = '../../data/train-jpegs/'
MAX_LENGTH = 256.

dset = Lungs(dicom_folders)
loader = DataLoader(dset, batch_size=1, shuffle=False, num_workers=0, collate_fn=lambda x: x)

In [351]:
# mkdir ./test_jpegs

In [352]:
# SAVEDIR = './test_jpegs'

In [353]:
# predictions = {}

In [11]:
resnet101 = models.resnet50()
fc_feat_size = resnet101.fc.in_features
resnet101.fc.in_features, resnet101.fc.out_features

(2048, 1000)

In [12]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    def __forward__(self, x):
        x = x.view(x.size(0), -1)
        return x
    
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

In [13]:
resnet101.avgpool = Identity()
resnet101.fc = Identity()

In [14]:
def weights_init(m):
    classname = m.__class__.__name__
    for l in m.modules():
        if isinstance(l, nn.Conv2d):
            torch.nn.init.uniform_(l.weight)
        elif isinstance(l, nn.BatchNorm2d):
            torch.nn.init.uniform_(l.weight)
        elif isinstance(l, nn.Linear):
            torch.nn.init.xavier_normal_(l.weight)
        
class MultiTaskHead(nn.Module):
    def __init__(self):
        super(MultiTaskHead, self).__init__()
        self.m = nn.Sequential(
                nn.Conv2d(2048, 512, kernel_size=(1,1), stride=(1,1), bias=False), 
                nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
                nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False),
                nn.BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.AdaptiveAvgPool2d(output_size=(1,1)),
        )
        self.l = nn.Sequential(
                nn.Linear(fc_feat_size, 1, bias=True), 
#                 nn.ReLU(),
#                 nn.Linear(512, 1), 
                nn.Sigmoid()
        )
        
    def forward(self, x):
        x = x.view(x.shape[0], 2048, int(np.sqrt(x.shape[1]/2048)), int(np.sqrt(x.shape[1]/2048)))
        x = self.m(x)
        x = self.l(x.squeeze())
        return x

class WrapperModel(nn.Module):
    def __init__(self, pretrained_model):
        super(WrapperModel, self).__init__()
        self.stage = 'train'
        
        self.backbone = pretrained_model
        '''
        image-level feature
        '''
        self.linear_pe_present_on_image = MultiTaskHead()
        weights_init(self.linear_pe_present_on_image)
        '''
        exam-level features
        '''
        self.linear_negative_exam_for_pe = MultiTaskHead()
        weights_init(self.linear_negative_exam_for_pe)
        
        self.linear_indeterminate = MultiTaskHead()
        weights_init(self.linear_indeterminate)
        
        self.linear_chronic_pe = MultiTaskHead()
        weights_init(self.linear_chronic_pe)
        
        self.linear_acute_and_chronic_pe = MultiTaskHead()
        weights_init(self.linear_acute_and_chronic_pe)
        
        self.linear_central_pe = MultiTaskHead()
        weights_init(self.linear_central_pe)
        
        self.linear_leftsided_pe = MultiTaskHead()
        weights_init(self.linear_leftsided_pe)
        
        self.linear_rightsided_pe = MultiTaskHead()
        weights_init(self.linear_rightsided_pe)
        
        self.linear_rv_lv_ratio_gte_1 = MultiTaskHead()
        weights_init(self.linear_rv_lv_ratio_gte_1)
        
        self.linear_rv_lv_ratio_lt_1 = MultiTaskHead()
        weights_init(self.linear_rv_lv_ratio_lt_1)
        
        '''
        informational features
        '''
        self.linear_qa_motion = MultiTaskHead()
        weights_init(self.linear_qa_motion)
        
        self.linear_qa_contrast = MultiTaskHead()
        weights_init(self.linear_qa_contrast)
                
        self.linear_true_filling_defect_not_pe = MultiTaskHead()
        weights_init(self.linear_true_filling_defect_not_pe)
        
        self.linear_flow_artifact = MultiTaskHead()
        weights_init(self.linear_flow_artifact)

    def forward(self, x):
#         if self.stage == 'train':
#             x, _ = self.backbone(x)
#         else: 
        x = self.backbone(x)
        
#         print('x1 .shape', x1.shape)
        x_pe_present_on_image = self.linear_pe_present_on_image(x)
        
        x_negative_exam_for_pe = self.linear_negative_exam_for_pe(x)
        x_indeterminate = self.linear_indeterminate(x)
        x_chronic_pe = self.linear_chronic_pe(x)
        x_acute_and_chronic_pe = self.linear_acute_and_chronic_pe(x)
        x_central_pe = self.linear_central_pe(x)
        x_leftsided_pe = self.linear_leftsided_pe(x)
        x_rightsided_pe = self.linear_rightsided_pe(x)
        x_rv_lv_ratio_gte_1 = self.linear_rv_lv_ratio_gte_1(x)
        x_rv_lv_ratio_lt_1 = self.linear_rv_lv_ratio_lt_1(x)
        
        x_qa_motion = self.linear_qa_motion(x)
        x_qa_contrast = self.linear_qa_contrast(x)
        x_true_filling_defect_not_pe = self.linear_true_filling_defect_not_pe(x)
        x_flow_artifact = self.linear_flow_artifact(x)
        
#         ys = []
#         for y_pred in [ x_pe_present_on_image, x_negative_exam_for_pe, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]:
#             ys.append(y_pred)
        ys = [x_pe_present_on_image, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]
#         for y_pred in [ x_pe_present_on_image, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]:
#             ys.append(y_pred)
        if self.stage == 'eval':
#             print(len(ys), ys[0].shape)
#             print(torch.cat(ys, 1).shape)
            return torch.cat(ys, 1).cpu().numpy()
        else:
            return torch.cat(ys, 1)
            

model = WrapperModel(resnet101).to(device)

In [ ]:
ls ../input/resnet-trained

In [15]:
model.load_state_dict(torch.load('../input/resnet-trained/resnet_2_9011964525914555_0_9047383336849932__0_3563135365031113_0_2710275558108921'))
model.stage='eval'
model.eval()

WrapperModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

In [ ]:
files

In [ ]:
sample_df.index[sample_df['id'].str.contains('37acdcf2ffe5')]

In [389]:
sample_df

,id,label
0,df06fad17bc3_negative_exam_for_pe,0.0
1,c8039e7f9e63_negative_exam_for_pe,0.5
2,761f6f1a9f5b_negative_exam_for_pe,0.5
3,c8db5b1f6b56_negative_exam_for_pe,0.5
4,462e805da1f1_negative_exam_for_pe,0.5
...,...,...
152698,5f34e0c61c00,0.5
152699,ccaa309b60da,0.5
152700,a274c8d0916e,0.5
152701,a702de2c99c6,0.5


In [ ]:
f.split('/')[-3]

In [ ]:
# sample_df.at[sample_df.index[sample_df['id'].str.contains(f[f.rfind('/')+1:-4])], 'label'] = [((y[[7,8,5,2,6,3,4,1]]>=0.5)).sum()] + y[[7,8,5,2,6,3,4,1]].tolist()
# sample_df.at[sample_df.index[sample_df['id'].str.contains('df06fad17bc3')], 'label']

# sample_df.at[sample_df.index[sample_df['id'].str.contains('df06fad17bc3')], 'label']
sample_df.loc[sample_df.index[sample_df['id'].str.contains("df06fad17bc3")], 'label'] = [((y[[7,8,5,2,6,3,4,1]]>=0.5)).sum()] + y[[7,8,5,2,6,3,4,1]].tolist()

In [ ]:
# # sample_df.at[sample_df.index[sample_df['id'].str.contains("df06fad17bc3")], 'label'] = [((value[[7,8,5,2,6,3,4,1]]>=0.5)).sum()] + value[[7,8,5,2,6,3,4,1]].tolist()

# for f in files:
# #     print(f[f.rfind('/')+1:-4])
#     print(files[0].split('/')[-3])
#     sample_df.loc[sample_df.index[sample_df['id'].str.contains(files[0].split('/')[-3])], 'label'] = [((y[[7,8,5,2,6,3,4,1]]>=0.5)).sum()] + y[[7,8,5,2,6,3,4,1]].tolist()

In [ ]:
# sample_df.loc[sample_df.id==files[0].split('/')[-1][:-4],  'label'] = 0
sample_df.loc[sample_df.id=='41cb110f177e',  'label']=0

In [377]:
y_pred[:,[7,8,5,2,6,3,4,1]].mean(0).shape

(8,)

In [362]:
sop_instance, study_instance

('0461e4e358cf', 'df06fad17bc3')

In [399]:
len(df.StudyInstanceUID.unique())*9

5850

In [397]:
len(df) + len(df.StudyInstanceUID.unique())*9

152703

In [403]:
files

array(['../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/0461e4e358cf.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/87264e009a34.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/011697fe8dd3.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/ec8da943ce56.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/2448585381f1.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/29e54c2f5e2c.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/9a68ee6b32e0.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/e5536a365e92.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/7dc3484f30fe.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/

In [400]:
files

array(['../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/0461e4e358cf.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/87264e009a34.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/011697fe8dd3.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/ec8da943ce56.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/2448585381f1.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/29e54c2f5e2c.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/9a68ee6b32e0.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/e5536a365e92.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/7dc3484f30fe.dcm',
       '../input/rsna-str-pulmonary-embolism-detection/

In [394]:
sample_df

,id,label
0,df06fad17bc3_negative_exam_for_pe,0.0
1,c8039e7f9e63_negative_exam_for_pe,0.5
2,761f6f1a9f5b_negative_exam_for_pe,0.5
3,c8db5b1f6b56_negative_exam_for_pe,0.5
4,462e805da1f1_negative_exam_for_pe,0.5
...,...,...
152698,5f34e0c61c00,0.5
152699,ccaa309b60da,0.5
152700,a274c8d0916e,0.5
152701,a702de2c99c6,0.5


In [451]:
[f.split('/')[-1][:-4] for f in files]

['0461e4e358cf',
 '87264e009a34',
 '011697fe8dd3',
 'ec8da943ce56',
 '2448585381f1',
 '29e54c2f5e2c',
 '9a68ee6b32e0',
 'e5536a365e92',
 '7dc3484f30fe',
 'e53da2a85896',
 '53d525cc93e6',
 '32f686697d5a',
 '6b79b3cf9733',
 '43409f59c805',
 '06c41b9f0ae3',
 'abcba4c8509d',
 'faaa6406111d',
 'ee697da02e09',
 '31238d9af74b',
 'fca5da292ace',
 'b77f8cb1019d',
 '3cfa1df17834',
 '0fe0832daf0a',
 'd0eefc7a108e',
 '5a87ce6ec2df',
 'd9b9fb36d5ec',
 '7e1be0d8fd1b',
 'a5680abfb4db',
 '78aa65812764',
 '061f216aff8e',
 'bb5851fc0817',
 '3a18ff60de58',
 'f1b9e7fdc760',
 'bb80fe6a719b',
 '1a7b57e2734a',
 '0e969006e5db',
 '3de23f56e1b1',
 '70782745f7c6',
 '4ac3a0733cb8',
 'a23a07a3ff06',
 '557d8caee6e7',
 'df01dfdca716',
 '25e3307ba7da',
 '9e6a68e27df0',
 'bdc531b699cd',
 'f8875f5ee1a0',
 'b502192e87ef',
 '749109870bc0',
 '65a8c955b812',
 '452a8285fb1c',
 '880016e5812e',
 'fbbf29b49f39',
 'f4e5bcde9926',
 'c12cad79b96c',
 '97d010db80c1',
 'e78de3dd1add',
 'b5f7c1016880',
 '8b42916c465c',
 '0899b81ece4e

In [470]:
c = [f.split('/')[-1][:-4] for f in files]

indices = [c.index(f) for f in sample_df[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files])].id.values.tolist()]

In [478]:
sample_df[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files])]

,id,label
5850,c3163725fcf6,0.5
5851,d54a8daaf836,0.5
5852,bdc531b699cd,0.5
5853,9e6a68e27df0,0.5
5854,25e3307ba7da,0.5
...,...,...
5989,452a8285fb1c,0.5
5990,65a8c955b812,0.5
5991,749109870bc0,0.5
5992,b502192e87ef,0.5


In [483]:
y_pred

array([[0.01851646, 0.0112947 , 0.00849619, ..., 0.04011171, 0.01408   ,
        0.06137199],
       [0.01660044, 0.01129422, 0.00804039, ..., 0.0414435 , 0.01402521,
        0.0627414 ],
       [0.01002916, 0.01129203, 0.00622499, ..., 0.04814875, 0.01377796,
        0.06941781],
       ...,
       [0.04070995, 0.01129931, 0.01277201, ..., 0.03153501, 0.01448246,
        0.05217823],
       [0.04280861, 0.0112996 , 0.01310826, ..., 0.03104872, 0.01450907,
        0.0516301 ],
       [0.03802878, 0.01129895, 0.01232911, ..., 0.03221101, 0.01444661,
        0.05292742]], dtype=float32)

In [489]:
((y_pred[indices,:]>=0.5).sum(1)>0).astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [475]:
[c[index] for index in indices]

['c3163725fcf6',
 'd54a8daaf836',
 'bdc531b699cd',
 '9e6a68e27df0',
 '25e3307ba7da',
 'df01dfdca716',
 '557d8caee6e7',
 'a23a07a3ff06',
 '4ac3a0733cb8',
 '70782745f7c6',
 '3de23f56e1b1',
 '0e969006e5db',
 'c64e321a9200',
 '1a7b57e2734a',
 'bb80fe6a719b',
 'f1b9e7fdc760',
 '3a18ff60de58',
 'bb5851fc0817',
 '061f216aff8e',
 '78aa65812764',
 'a5680abfb4db',
 '7e1be0d8fd1b',
 'd9b9fb36d5ec',
 '8c562044f220',
 '5a87ce6ec2df',
 'd0eefc7a108e',
 '0fe0832daf0a',
 '3cfa1df17834',
 'b77f8cb1019d',
 'fca5da292ace',
 '31238d9af74b',
 'ee697da02e09',
 'faaa6406111d',
 'abcba4c8509d',
 'c6b826a3831d',
 '06c41b9f0ae3',
 '43409f59c805',
 '6b79b3cf9733',
 '32f686697d5a',
 '53d525cc93e6',
 'e53da2a85896',
 '7dc3484f30fe',
 'e5536a365e92',
 '9a68ee6b32e0',
 '29e54c2f5e2c',
 '0bc89a034a61',
 '2448585381f1',
 'ec8da943ce56',
 '011697fe8dd3',
 '87264e009a34',
 '0461e4e358cf',
 '00e626c5a0e3',
 'e61807f376c0',
 '37acdcf2ffe5',
 'f22069caec34',
 'fd6c44d39e48',
 'ab7c8b84d912',
 'fccacbfdb452',
 'fb28971b0bf6

In [465]:
# sample_df['id']==[f.split('/')[-1][:-4] for f in files]
# # sample_df[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files])]
# [f.split('/')[-1][:-4] for f in files].index(sample_df[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files])].id.values.tolist())
# [L[i] for i in Idx]
c = [f.split('/')[-1][:-4] for f in files]
[c.index(f) f for f in sample_df[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files])].id.values.tolist()]

# sample_df['id']==[f.split('/')[-1][:-4] for f in files]
# # sample_df[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files])]
# [f.split('/')[-1][:-4] for f in files].index(sample_df[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files])].id.values.tolist())
# [L[i] for i in Idx]

[c.index(f) for f in sample_df[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files])].id.values.tolist()]

NameError: name 'c' is not defined

In [443]:
# sample_df.loc[sample_df.id==[f.split('/')[-1][:-4] for f in files]]
# sample_df.loc[sample_df.id==[f.split('/')[-1][:-4] for f in files]]
# ind_ = sample_df.index[sample_df.id==files[0].split('/')[-1][:-4]].tolist()[0]
# sample_df.loc[ind_:ind_+len(files)].id
# sample_df.loc[sample_df.id==[f.split('/')[-1][:-4] for f in files]]

sample_df.at[sample_df['id']==[f.split('/')[-1][:-4] for f in files]]

ValueError: ('Lengths must match to compare', (152703,), (144,))

In [434]:
sample_df.index[sample_df.id==files[0].split('/')[-1][:-4]].tolist()[0]

5900

In [500]:
c = [f.split('/')[-1][:-4] for f in files]
indices = [c.index(f) for f in sample_df[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files])].id.values.tolist()]
print([c[i] for i in indices])

['c3163725fcf6', 'd54a8daaf836', 'bdc531b699cd', '9e6a68e27df0', '25e3307ba7da', 'df01dfdca716', '557d8caee6e7', 'a23a07a3ff06', '4ac3a0733cb8', '70782745f7c6', '3de23f56e1b1', '0e969006e5db', 'c64e321a9200', '1a7b57e2734a', 'bb80fe6a719b', 'f1b9e7fdc760', '3a18ff60de58', 'bb5851fc0817', '061f216aff8e', '78aa65812764', 'a5680abfb4db', '7e1be0d8fd1b', 'd9b9fb36d5ec', '8c562044f220', '5a87ce6ec2df', 'd0eefc7a108e', '0fe0832daf0a', '3cfa1df17834', 'b77f8cb1019d', 'fca5da292ace', '31238d9af74b', 'ee697da02e09', 'faaa6406111d', 'abcba4c8509d', 'c6b826a3831d', '06c41b9f0ae3', '43409f59c805', '6b79b3cf9733', '32f686697d5a', '53d525cc93e6', 'e53da2a85896', '7dc3484f30fe', 'e5536a365e92', '9a68ee6b32e0', '29e54c2f5e2c', '0bc89a034a61', '2448585381f1', 'ec8da943ce56', '011697fe8dd3', '87264e009a34', '0461e4e358cf', '00e626c5a0e3', 'e61807f376c0', '37acdcf2ffe5', 'f22069caec34', 'fd6c44d39e48', 'ab7c8b84d912', 'fccacbfdb452', 'fb28971b0bf6', 'fb3928451579', 'f473c933fb03', '95375959d136', '9f9f52

In [511]:
sample_df.at[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files]), 'label'] = [i for i in range(144)]

In [515]:
sample_df.loc[sample_df['id'].isin([f.split('/')[-1][:-4] for f in files]), 'label']

5850      0.0
5851      1.0
5852      2.0
5853      3.0
5854      4.0
        ...  
5989    139.0
5990    140.0
5991    141.0
5992    142.0
5993    143.0
Name: label, Length: 144, dtype: float64

In [520]:
# c = [f.split('/')[-1][:-4] for f in files]
# indices = [c.index(f) for f in sample_df[sample_df['id'].isin(c)].id.values.tolist()]
# sample_df.at[sample_df['id'].isin(c), 'label'] = ((y_pred[indices,:]>=0.5).sum(1)>0).astype(int).tolist()

In [16]:
with torch.no_grad():
    for i, data in enumerate(tqdm(loader, total=len(loader))):
        data = data[0]
        if type(data) == type(None): continue
#         try:
        image, files = data
        # Windows from https://pubs.rsna.org/doi/pdf/10.1148/rg.245045008
        image_lung = np.expand_dims(window(image, WL=-600, WW=1500), axis=3)
        image_mediastinal = np.expand_dims(window(image, WL=40, WW=400), axis=3)
        image_pe_specific = np.expand_dims(window(image, WL=100, WW=700), axis=3)
        image = np.concatenate([image_mediastinal, image_pe_specific, image_lung], axis=3)
        rat = MAX_LENGTH / np.max(image.shape[1:])
        image = zoom(image, [1.,rat,rat,1.], prefilter=False, order=1)

        images = [test_ds_trans(x_) for x_ in image]
        x = torch.stack(images).to(device)
        y_pred = model(x)
        study_instance = files[0].split('/')[-3]
        
        c = [f.split('/')[-1][:-4] for f in files]
        indices = [c.index(f) for f in sample_df[sample_df['id'].isin(c)].id.values.tolist()]
        sample_df.at[sample_df['id'].isin(c), 'label'] = ((y_pred[indices,:]>=0.5).sum(1)>0).astype(int).tolist()
        sample_df.loc[sample_df.index[sample_df['id'].str.contains(study_instance)], 'label'] = [((y_pred[[7,8,5,2,6,3,4,1]]>=0.5)).sum()] + y_pred[[7,8,5,2,6,3,4,1]].tolist()

  0%|          | 0/650 [00:10<?, ?it/s]


NameError: name 'y' is not defined

In [527]:
save_obj(sample_df, './sample_df')
from IPython.display import FileLink
FileLink(r'./sample_df.pkl')

/kaggle/working/sample_df.pkl

In [ ]:
df.StudyInstanceUID.unique()

In [ ]:
# def save_obj(obj, name ):
#     with open(name + '.pkl', 'wb') as f:
#         pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

# def load_obj(name ):
#     with open(name + '.pkl', 'rb') as f:
#         return pickle.load(f)

In [ ]:
# save_obj(predictions, './predictions')

In [ ]:
# save_obj(predictions, './predictions')
# from IPython.display import FileLink
# FileLink(r'./predictions.pkl')

In [ ]:
!ls ../input/

In [ ]:
predicted = load_obj('../input/predicted-with-resnet-50-dict/predictions')

In [ ]:
predicted

In [ ]:
len(predicted)

In [ ]:
df.shape

In [ ]:
predicted['../input/rsna-str-pulmonary-embolism-detection/test/df06fad17bc3/857e3d760445/2448585381f1.dcm']

In [ ]:
negative_pes=[]

In [ ]:
for study in df.StudyInstanceUID.unique():
    study_values = np.array([value for key, value in predicted.items() if study in key.lower()])
#     print(study, study_values.shape)
    if (study_values >= 0.5).sum() ==0:
        
#         negative_pes.append(study)

In [ ]:
# sample_df[sample_df.id=='df06fad17bc3_negative_exam_for_pe']

len(negative_pes)

In [ ]:
# [x_pe_present_on_image, x_indeterminate, x_chronic_pe, x_acute_and_chronic_pe, x_central_pe, x_leftsided_pe, x_rightsided_pe, x_rv_lv_ratio_gte_1, x_rv_lv_ratio_lt_1, x_qa_motion, x_qa_contrast, x_true_filling_defect_not_pe, x_flow_artifact]

In [ ]:
sample_df

In [ ]:
df

In [ ]:
sample_df = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/sample_submission.csv')

In [ ]:
len(predicted) + len(df.StudyInstanceUID.unique())*9

In [ ]:
sample_df

In [ ]:
features_df = ['negative_exam_for_pe', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1', 'leftsided_pe', 'chronic_pe', 'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate']
features_model = ['x_pe_present_on_image', 'indeterminate', 'chronic_pe', 'acute_and_chronic_pe', 'central_pe', 'leftsided_pe', 'rightsided_pe', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1', 'qa_motion', 'qa_contrast', 'true_filling_defect_not_pe', 'flow_artifact']

In [ ]:
# sample_df.loc[len(df.StudyInstanceUID.unique())*8:9*len(df.StudyInstanceUID.unique())+1]

In [ ]:
value

In [ ]:
[((value[[7,8,5,2,6,3,4,1]]>=0.5)).sum()] + value[[7,8,5,2,6,3,4,1]].tolist()

In [ ]:
# sample_df.loc[sample_df.id == '41cb110f177e']
sample_df[sample_df['id'].str.contains("df06fad17bc3")]

In [ ]:
sample_df.at[sample_df.index[sample_df['id'].str.contains("df06fad17bc3")], 'label'] = [((value[[7,8,5,2,6,3,4,1]]>=0.5)).sum()] + value[[7,8,5,2,6,3,4,1]].tolist()

In [ ]:
# for index, (key, value) in tqdm(enumerate(predicted.items()), total=len(predicted)):
#     key.split('/')[-1]
for index, (key, value) in tqdm(enumerate(predicted.items())):
    l = key.split('/')
    studyInstance = l[-3]
#     seriesInstance = l[-2]
#     imageInstance = l[-1][:-4]
#     print(studyInstance, seriesInstance, imageInstance, value.shape)
    sample_df.at[sample_df.index[sample_df['id'].str.contains(studyInstance)], 'label'] = [((value[[7,8,5,2,6,3,4,1]]>=0.5)).sum()] + value[[7,8,5,2,6,3,4,1]].tolist()
#     print(sample_df[imageInstance in sample_df.id])
#     break
#     print(studyInstance in sample_df.loc[index].id)
#     if not studyInstance in sample_df.loc[index].id:
#         print('AAAAAAAA', index, studyInstance, sample_df.loc[index])
#         break

In [ ]:
for index, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):
    if 'negative' in row.id:
#         print(row.id.split('_')[0])
        study_values = np.array([value for key, value in predicted.items() if row.id.split('_')[0] in key.lower()])
        print(row.id, )
        if (study_values>=0.5).sum():
            sample_df.at[index, row.id] = 1
#             print(sample_df.loc[index])
        else:
            sample_df.at[index, row.id] = 0
#             print(sample_df.loc[index])
            
    else:
        break

In [ ]:
sample_df